In [ ]:
!pip install osmnx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import math

In [ ]:
import networkx as nx
import osmnx as ox
from collections import Counter

%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#load infrared data
data = pd.read_csv('/content/drive/MyDrive/AIA STUDIO/1. DATASETS/new_datasets_Sophie/Collab Maria/infrared_SR_1_header.csv')
data

,u,v,key,level,bbox,time,radiation
0,84295307,1374661520,0,0,0,0:01:50,0.382716
1,84295307,429583975,0,0,0,0:01:50,0.316306
2,84295307,2838285959,0,0,0,0:01:50,0.371836
3,84295307,2838285959,0,1,0,0:01:50,0.365164
4,129531323,3056377534,0,0,0,0:01:50,0.243435
...,...,...,...,...,...,...,...
313816,8326287330,8326287311,0,0,900,1:42:11,0.394177
313817,8326287330,8326287329,0,0,900,1:42:11,0.400062
313818,8326287330,8326287322,0,0,900,1:42:11,0.395362
313819,8326287330,8326287322,0,1,900,1:42:11,0.401866


In [ ]:
#make u,v,key values the index of dataframe
index = list(zip(data['u'], data['v'], data['key'])) 
data['index'] = index
data = data.set_index('index')
data

,u,v,key,level,bbox,time,radiation
index,,,,,,,
"(84295307, 1374661520, 0)",84295307,1374661520,0,0,0,0:01:50,0.382716
"(84295307, 429583975, 0)",84295307,429583975,0,0,0,0:01:50,0.316306
"(84295307, 2838285959, 0)",84295307,2838285959,0,0,0,0:01:50,0.371836
"(84295307, 2838285959, 0)",84295307,2838285959,0,1,0,0:01:50,0.365164
"(129531323, 3056377534, 0)",129531323,3056377534,0,0,0,0:01:50,0.243435
...,...,...,...,...,...,...,...
"(8326287330, 8326287311, 0)",8326287330,8326287311,0,0,900,1:42:11,0.394177
"(8326287330, 8326287329, 0)",8326287330,8326287329,0,0,900,1:42:11,0.400062
"(8326287330, 8326287322, 0)",8326287330,8326287322,0,0,900,1:42:11,0.395362


In [ ]:
#load street data
edges = pd.read_csv('/content/drive/MyDrive/AIA STUDIO/RESULTS/final dataset/FINAL_DATASET.csv')
edges

,Unnamed: 0,u,v,key,osmid,oneway,name,highway,maxspeed,length,...,short term parking,parking_places,waterinfluence,grassinfluence,Street materials,UHI,DIRhori,GLO_hori_d,GLO_hori_real,SSD_sum
0,0,199732,2447309494,0,29065909,0,Universitätsring,2,50,11.008,...,0,0.0,0.267570,0.883517,1.0,0.405181,4.023,6.974,6.963,13.0
1,1,199732,318697385,0,25535788,0,Rathausplatz,3,50,20.372,...,0,0.0,0.160478,0.863910,1.0,0.405181,4.023,6.974,6.963,13.0
2,2,199732,2447277599,0,537176980,0,Dr.-Karl-Renner-Ring,2,50,22.643,...,0,0.0,0.127268,1.009087,1.0,0.405181,4.023,6.974,6.963,13.0
3,3,2447309494,3653696701,0,29065909,0,Universitätsring,2,50,13.511,...,0,0.0,0.447723,0.739094,1.0,0.405181,4.023,6.974,6.963,13.0
4,4,2447309494,199732,0,29065909,0,Universitätsring,2,50,11.008,...,0,0.0,0.267570,0.883517,1.0,0.405181,4.023,6.974,6.963,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298929,298929,9817093021,9816987805,0,1069575181,0,0,5,0,35.565,...,0,0.0,0.000000,0.000000,0.0,0.000000,0.000,0.000,0.000,0.0
298930,298930,9817093055,9817093055,0,"[1069575184, 1069575186, 1069575191]",0,0,3,0,100.214,...,0,0.0,0.000000,0.000000,0.0,0.000000,0.000,0.000,0.000,0.0
298931,298931,9817093055,9817093055,1,"[1069575184, 1069575186, 1069575191]",0,0,3,0,100.214,...,0,0.0,0.000000,0.000000,0.0,0.000000,0.000,0.000,0.000,0.0
298932,298932,9817093055,9817093049,0,"[1069575185, 1069575188, 1069575190]",0,0,3,0,29.206,...,0,0.0,0.000000,0.000000,0.0,0.000000,0.000,0.000,0.000,0.0


In [ ]:
#make the u,v,key values the index of the dataframe
edgeIndex = list(zip(edges['u'], edges['v'], edges['key']))
edges['index'] = edgeIndex
edges = edges.set_index('index')
edges

,Unnamed: 0,u,v,key,osmid,oneway,name,highway,maxspeed,length,...,short term parking,parking_places,waterinfluence,grassinfluence,Street materials,UHI,DIRhori,GLO_hori_d,GLO_hori_real,SSD_sum
index,,,,,,,,,,,,,,,,,,,,,
"(199732, 2447309494, 0)",0,199732,2447309494,0,29065909,0,Universitätsring,2,50,11.008,...,0,0.0,0.267570,0.883517,1.0,0.405181,4.023,6.974,6.963,13.0
"(199732, 318697385, 0)",1,199732,318697385,0,25535788,0,Rathausplatz,3,50,20.372,...,0,0.0,0.160478,0.863910,1.0,0.405181,4.023,6.974,6.963,13.0
"(199732, 2447277599, 0)",2,199732,2447277599,0,537176980,0,Dr.-Karl-Renner-Ring,2,50,22.643,...,0,0.0,0.127268,1.009087,1.0,0.405181,4.023,6.974,6.963,13.0
"(2447309494, 3653696701, 0)",3,2447309494,3653696701,0,29065909,0,Universitätsring,2,50,13.511,...,0,0.0,0.447723,0.739094,1.0,0.405181,4.023,6.974,6.963,13.0
"(2447309494, 199732, 0)",4,2447309494,199732,0,29065909,0,Universitätsring,2,50,11.008,...,0,0.0,0.267570,0.883517,1.0,0.405181,4.023,6.974,6.963,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(9817093021, 9816987805, 0)",298929,9817093021,9816987805,0,1069575181,0,0,5,0,35.565,...,0,0.0,0.000000,0.000000,0.0,0.000000,0.000,0.000,0.000,0.0
"(9817093055, 9817093055, 0)",298930,9817093055,9817093055,0,"[1069575184, 1069575186, 1069575191]",0,0,3,0,100.214,...,0,0.0,0.000000,0.000000,0.0,0.000000,0.000,0.000,0.000,0.0
"(9817093055, 9817093055, 1)",298931,9817093055,9817093055,1,"[1069575184, 1069575186, 1069575191]",0,0,3,0,100.214,...,0,0.0,0.000000,0.000000,0.0,0.000000,0.000,0.000,0.000,0.0


In [ ]:
edges['radiation'] = [math.nan]*len(edges.index)
edges

,Unnamed: 0,u,v,key,osmid,oneway,name,highway,maxspeed,length,...,parking_places,waterinfluence,grassinfluence,Street materials,UHI,DIRhori,GLO_hori_d,GLO_hori_real,SSD_sum,radiation
index,,,,,,,,,,,,,,,,,,,,,
"(199732, 2447309494, 0)",0,199732,2447309494,0,29065909,0,Universitätsring,2,50,11.008,...,0.0,0.267570,0.883517,1.0,0.405181,4.023,6.974,6.963,13.0,NaN
"(199732, 318697385, 0)",1,199732,318697385,0,25535788,0,Rathausplatz,3,50,20.372,...,0.0,0.160478,0.863910,1.0,0.405181,4.023,6.974,6.963,13.0,NaN
"(199732, 2447277599, 0)",2,199732,2447277599,0,537176980,0,Dr.-Karl-Renner-Ring,2,50,22.643,...,0.0,0.127268,1.009087,1.0,0.405181,4.023,6.974,6.963,13.0,NaN
"(2447309494, 3653696701, 0)",3,2447309494,3653696701,0,29065909,0,Universitätsring,2,50,13.511,...,0.0,0.447723,0.739094,1.0,0.405181,4.023,6.974,6.963,13.0,NaN
"(2447309494, 199732, 0)",4,2447309494,199732,0,29065909,0,Universitätsring,2,50,11.008,...,0.0,0.267570,0.883517,1.0,0.405181,4.023,6.974,6.963,13.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(9817093021, 9816987805, 0)",298929,9817093021,9816987805,0,1069575181,0,0,5,0,35.565,...,0.0,0.000000,0.000000,0.0,0.000000,0.000,0.000,0.000,0.0,NaN
"(9817093055, 9817093055, 0)",298930,9817093055,9817093055,0,"[1069575184, 1069575186, 1069575191]",0,0,3,0,100.214,...,0.0,0.000000,0.000000,0.0,0.000000,0.000,0.000,0.000,0.0,NaN
"(9817093055, 9817093055, 1)",298931,9817093055,9817093055,1,"[1069575184, 1069575186, 1069575191]",0,0,3,0,100.214,...,0.0,0.000000,0.000000,0.0,0.000000,0.000,0.000,0.000,0.0,NaN


In [ ]:
radiation = []
for i in range(len(data.index)):
  try:
    edges.loc[[data.index[i]],'radiation'] = data.iloc[i]['radiation']*1400
  except Exception:
    radiation.append(math.nan)

In [ ]:
edges.to_csv('/content/drive/MyDrive/AIA STUDIO/1. DATASETS/new_datasets_Sophie/Collab Maria/FINAL_DATASET_withSR.csv')